# Setup

In [1]:
import pandas as pd
import datetime
from wordcloud import WordCloud, STOPWORDS 
import sys
from os import path
import numpy as np
from PIL import Image
from textblob import TextBlob
import nltk
import re
import matplotlib
import matplotlib.pyplot as plt

# Import Data

In [41]:
df = pd.read_csv("sms_scan_exit_survey.csv",keep_default_na=False) #leave blanks as blanks

###  dates take the place of subject labels
### Phrasing of question was changed after first six participants, much messier individual sequence responses for those first subjects, so I can either spend a bunch of time trying to organize that, or ignore those subjects, or just ignore individual sequences and merge across questions

In [45]:
df.head()

,Subject,Q1,Q2,Q3,Q4,Q5,Q6,Q7
0,2017/07/26 2:14:42 PM MDT,,,"Yes, I thought of a singular event that the wo...",I didn't try to think of any similar events th...,"No, they were more like short segments sewed t...",,
1,2017/07/28 12:50:17 PM MDT,,,2/intact - no particular event\n3/scrambled - ...,"2/intact - yes very much, as this would be a c...","3/scrambled - no, this event in my head lasted...",No,No
2,2017/08/02 10:58:33 AM MDT,,,2 scrambled no\n3 random1 no\n4 intact no\n5 s...,2 scrambled yes\n3 random1 no\n4 intact no\n5 ...,2 scrambled no\n3 random1 yes\n4 intact yes\n5...,No problems.,I think that the problems I had were there on ...
3,2017/08/24 2:48:51 PM MDT,,,no for all,"yes for 2, 5, 6, 7, 9, 12, 13, 15 and no for t...","yes for 3, 4, 16 (kind of) and no for the rest",,"2, intact -- I realized that it was a basic d..."
4,2017/08/30 2:45:52 PM MDT,,,intact - 2 - No \n\nscramble - 3 - Yes; I ...,intact - 2 - Yes. I thought of my own experi...,intact - 2 - No\n\nscramble - 3 - No\n\nran...,No,No


In [51]:
# rename columns
df.columns = ['Subject','Q1','Q2','Q3','Q4','Q5','Q6','Q7']
# merge across questions, so all responses from each subject will be in one column
df['merged'] =  df.iloc[:, 1:].sum(axis=1)

# Look for between subject overall differences